**Dual Stage Attention SpyNet (DAS)**

In [ ]:
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Input, Add, BatchNormalization, Activation, Multiply, GlobalAveragePooling2D, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

# CBAM Attention Module
def cbam_block(input_tensor, reduction_ratio=16):
    """ CBAM (Convolutional Block Attention Module) """

    # Channel Attention
    channel = GlobalAveragePooling2D()(input_tensor)
    channel = Reshape((1, 1, input_tensor.shape[-1]))(channel)
    channel = Conv2D(input_tensor.shape[-1] // reduction_ratio, (1, 1), activation='relu', padding='same')(channel)
    channel = Conv2D(input_tensor.shape[-1], (1, 1), activation='sigmoid', padding='same')(channel)
    channel_att = Multiply()([input_tensor, channel])

    # Spatial Attention
    spatial = Conv2D(1, (7, 7), activation='sigmoid', padding='same')(channel_att)
    spatial_att = Multiply()([channel_att, spatial])

    return spatial_att

# SpyNet Block
def spynet_block(input_tensor):
    """ Basic SpyNet Block with CBAM Attention """
    x = Conv2D(64, (7, 7), padding="same", activation="relu")(input_tensor)
    x = Conv2D(64, (5, 5), padding="same", activation="relu")(x)
    x = cbam_block(x)  # CBAM Added Here
    x = Conv2D(32, (3, 3), padding="same", activation="relu")(x)
    x = Conv2D(2, (3, 3), padding="same")(x)  # Output Optical Flow (Vx, Vy)
    return x

# DAS Model (SpyNet + CBAM)
def build_das_model():
    """ DAS Model using SpyNet and CBAM Attention """

    # Input: Two consecutive frames (Stacked)
    input_layer = Input(shape=(128, 128, 2))

    # SpyNet with CBAM
    output_layer = spynet_block(input_layer)

    # Build Model
    model = Model(inputs=input_layer, outputs=output_layer, name="DAS_SpyNet_CBAM")
    return model

# Create and save the model
das_model = build_das_model()
das_model.compile(optimizer="adam", loss="mse")  # Loss for optical flow
das_model.save("models/das_spynet_cbam.h5")  # Save model

# Load dataset function
def load_data(im_dir):
    all_images = []
    list_of_files = sorted(os.listdir(im_dir), reverse=False)

    for im_folder in list_of_files:  # Iterate over each folder
        list_of_img_files = sorted(os.listdir(os.path.join(im_dir, im_folder)), reverse=False)

        for image_file in list_of_img_files:  # Iterate over each image
            image_path = os.path.join(im_dir, im_folder, image_file)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load grayscale image
            image = cv2.resize(image, (128, 128))  # Resize to 128x128
            all_images.append([image])

    all_images = np.array(all_images).reshape(-1, 20, 128, 128, 1)  # Reshape for model
    return all_images

# Load DSAPM model
dsapm_model = load_model("models/dsapm.h5")

# Load DAS (SpyNet + CBAM) model
das_model = load_model("models/das_spynet_cbam.h5")

# Load dataset (Test data from Google Drive)
dataset_path = "/content/drive/MyDrive/Test_Data"
test_data = load_data(dataset_path)

# Training Data used to generate threshold
train_data = []
all_images = load_data("/content/drive/MyDrive/Train_Data")  # DSAPM-trained dataset

for sequence in all_images:  # Use all sequences
    predicted_frames = []
    for i in range(18):
        Z_n = sequence[i, ..., 0]
        predicted_Z_n1 = dsapm_model.predict(np.expand_dims(sequence[i:i+1], axis=0))
        predicted_frames.append(predicted_Z_n1)
    train_data.append(predicted_frames)

train_data = np.array(train_data)


# Compute End-Point Error (EPE)
def compute_epe(predicted_flow, gt_flow):
    return np.sqrt(np.sum((predicted_flow - gt_flow) ** 2, axis=-1)).mean()

# Compute flow score
def compute_flow_score(epe_values):
    epe_min, epe_max = np.min(epe_values), np.max(epe_values)
    return (epe_values - epe_min) / (epe_max - epe_min)

# Train DAS (SpyNet + CBAM) and compute threshold
normal_epe_values = []
for sequence in train_data:
    for i in range(17):  # Using first 17 frames
        Z_n = sequence[i, ..., 0]
        Z_n1 = sequence[i+1, ..., 0]

        # Compute ground truth optical flow using DAS (SpyNet + CBAM)
        V_gt = das_model.predict(np.expand_dims(np.stack([Z_n, Z_n1], axis=-1), axis=0))

        # Compute predicted optical flow using DAS
        V_L = das_model.predict(np.expand_dims(np.stack([Z_n, sequence[i+1, ..., 0]], axis=-1), axis=0))

        epe = compute_epe(V_L, V_gt)
        normal_epe_values.append(epe)

flow_scores = compute_flow_score(np.array(normal_epe_values))
Fl_th = np.max(flow_scores)  # Threshold

# Test phase
for sequence in test_data:
    for i in range(18):
        Z_n = sequence[i, ..., 0]
        Z_n1 = sequence[i+1, ..., 0]

        # Compute ground truth and predicted optical flow using DAS (SpyNet + CBAM)
        V_gt = das_model.predict(np.expand_dims(np.stack([Z_n, Z_n1], axis=-1), axis=0))
        V_L = das_model.predict(np.expand_dims(np.stack([Z_n, sequence[i+1, ..., 0]], axis=-1), axis=0))

        epe = compute_epe(V_L, V_gt)
        flow_score = (epe - np.min(normal_epe_values)) / (np.max(normal_epe_values) - np.min(normal_epe_values))

        if flow_score > Fl_th:
            print("Anomaly detected at frame", i)

        # Visualize optical flow
        hsv = np.zeros((128, 128, 3), dtype=np.uint8)
        hsv[..., 1] = 255
        mag, ang = cv2.cartToPolar(V_L[..., 0], V_L[..., 1])
        hsv[..., 0] = ang * 180 / np.pi / 2
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
        flow_rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        cv2.imshow("Optical Flow", flow_rgb)
        cv2.waitKey(1)